In [1]:
!ls ~/datasets/astera_hetero

processed  raw


In [2]:
from datatype_recovery.models.heteromodels import DragonHGT

model = DragonHGT.create_model(confidence=True, heads=3)
model

DragonHGT(
  (gnn_layers): ModuleList(
    (0-4): 5 x HGTConv(-1, 192, heads=3)
  )
  (shared_linear): Sequential(
    (0): Linear(in_features=192, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=64, bias=True)
    (3): ReLU()
    (4): Linear(in_features=64, out_features=64, bias=True)
    (5): ReLU()
  )
  (ptr_l1_head): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=3, bias=True)
  )
  (ptr_l2_head): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=3, bias=True)
  )
  (ptr_l3_head): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_features=3, bias=True)
  )
  (leaf_category_head): Sequential(
    (0): Linear(in_features=64, out_features=64, bias=True)
    (1): ReLU()
    (2): Linear(in_features=64, out_

In [3]:
from pathlib import Path
from datatype_recovery.models.dataset import SimpleTypeDataset, load_dataset_from_path
from datatype_recovery.models.dataset.encoding import HeteroNodeEncoder
from torch_geometric.loader import DataLoader

# dataset = SimpleTypeDataset(Path.home()/'datasets/astera_hetero')
dataset = load_dataset_from_path(Path.home()/'datasets/astera_hetero2')
dataset

SimpleTypeDataset(300)

In [4]:
HeteroNodeEncoder.get_metadata()[1]

[('Default', 'Param3', 'Default'),
 ('Default', 'Param2', 'Default'),
 ('Default', 'Param4', 'Default'),
 ('Default', 'Param1', 'Default'),
 ('Default', 'Left', 'Default'),
 ('Default', 'Right', 'Default'),
 ('Default', 'ArrayIdxExpr', 'Default'),
 ('Default', 'ArrayVar', 'Default'),
 ('Default', 'Default', 'Default')]

In [19]:
dataset[0].to_homogeneous()#.to_heterogeneous()

Data(y=[1, 22], varid=[4], num_other_vars=2, edge_index=[2, 44], x=[22, 64], node_type=[22], edge_type=[44])

In [13]:
ht = load_dataset_from_path(Path.home()/'datasets/homo_train')
ht[0]

Data(x=[7, 86], edge_index=[2, 12], edge_attr=[12, 26], varid=[4], num_other_vars=[1], y=[1, 22])

In [14]:
ht[0].to_heterogeneous()

HeteroData(
  varid=[4],
  num_other_vars=[1],
  y=[1, 22],
  0={ x=[7, 86] },
  (0, 0, 0)={
    edge_index=[2, 12],
    edge_attr=[12, 26],
  }
)

In [19]:
from datatype_recovery.models.dataset.encoding import NodeEncoder
set([NodeEncoder().decode_node(d.x_dict['Default'][0])[0] for d in dataset])    # looks like we're still good on DeclRefExpr's being node 0 :)

{'DeclRefExpr'}

In [4]:
# data_list = dataset.generate_data_from_vars()
data_list = list(dataset)

In [5]:
data_list[232]

HeteroData(
  varid=[4],
  num_other_vars=4,
  y=[1, 22],
  NodeWithType={ x=[11, 53] },
  Operator={ x=[4, 64] },
  CallExpr={ x=[2, 141] },
  Default={ x=[2, 31] },
  (Operator, Left, NodeWithType)={ edge_index=[2, 4] },
  (Operator, Right, CallExpr)={ edge_index=[2, 2] },
  (CallExpr, Default, NodeWithType)={ edge_index=[2, 2] },
  (CallExpr, Param1, Default)={ edge_index=[2, 1] },
  (Default, ArrayVar, NodeWithType)={ edge_index=[2, 2] },
  (Default, ArrayIdxExpr, NodeWithType)={ edge_index=[2, 2] },
  (CallExpr, Param3, NodeWithType)={ edge_index=[2, 1] },
  (CallExpr, Param1, NodeWithType)={ edge_index=[2, 1] },
  (CallExpr, Param2, Default)={ edge_index=[2, 1] },
  (Operator, Right, NodeWithType)={ edge_index=[2, 1] },
  (Operator, Right, Operator)={ edge_index=[2, 2] },
  (NodeWithType, rev_Left, Operator)={ edge_index=[2, 4] },
  (CallExpr, rev_Right, Operator)={ edge_index=[2, 2] },
  (NodeWithType, rev_Default, CallExpr)={ edge_index=[2, 2] },
  (Default, rev_Param1, CallExp

In [6]:
hbatch = list(DataLoader(data_list, batch_size=4))[0]
batch_dict = hbatch.batch_dict
batch_dict

{'NodeWithType': tensor([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3]),
 'Operator': tensor([0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 3, 3, 3]),
 'Literal': tensor([0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 2, 3]),
 'Default': tensor([0, 0, 1, 1, 2, 3])}

In [7]:
from datatype_recovery.models.structural_model import get_node0_indices
print(batch_dict['NodeWithType'])
n0_idxs = get_node0_indices(batch_dict['NodeWithType'])
n0_idxs

tensor([0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3])


[0, 7, 14, 19]

In [8]:
from datatype_recovery.models.dataset.encoding import *
nwt = hbatch.x_dict['NodeWithType']
kinds = [NodeKinds.decode(x) for x in nwt]
print([(i, kinds[i]) for i in n0_idxs])
# kinds

[(0, 'DeclRefExpr'), (7, 'DeclRefExpr'), (14, 'DeclRefExpr'), (19, 'DeclRefExpr')]


In [17]:
model.train()
out, conf = model(hbatch.x_dict, hbatch.edge_index_dict, hbatch.batch_dict)

tensor([[0.0744],
        [0.0744],
        [0.0744],
        [0.0744]], grad_fn=<AddmmBackward0>)

In [74]:
#hbatch.x_dict['Default'].shape
out['Default'].shape

torch.Size([6, 64])

In [7]:
dobj, slices = dataset.collate(data_list)

In [8]:
shapes = []
for k, v in slices.items():
    if isinstance(v, dict):
        if 'x' in v:
            shapes.append((k, v['x'].shape))
        elif 'edge_index' in v:
            shapes.append((k, v['edge_index'].shape))
        else:
            print(v.keys())
    else:
        shapes.append((k, v.shape))

In [9]:
shapes

[('varid', torch.Size([301])),
 ('num_other_vars', torch.Size([301])),
 ('y', torch.Size([301])),
 ('NodeWithType', torch.Size([301])),
 ('Operator', torch.Size([300])),
 ('Literal', torch.Size([277])),
 ('Default', torch.Size([226])),
 (('Operator', 'Left', 'NodeWithType'), torch.Size([291])),
 (('Operator', 'Right', 'Literal'), torch.Size([225])),
 (('Operator', 'Right', 'Operator'), torch.Size([146])),
 (('NodeWithType', 'Default', 'NodeWithType'), torch.Size([133])),
 (('Default', 'Default', 'Operator'), torch.Size([169])),
 (('NodeWithType', 'Default', 'Default'), torch.Size([154])),
 (('NodeWithType', 'rev_Left', 'Operator'), torch.Size([291])),
 (('Literal', 'rev_Right', 'Operator'), torch.Size([225])),
 (('Operator', 'rev_Default', 'Default'), torch.Size([169])),
 (('Default', 'rev_Default', 'NodeWithType'), torch.Size([154]))]

In [18]:
dobj.edge_index_dict[('Operator','Right','Operator')].shape
dobj.edge_index_dict[('Operator','Right','Operator')]

tensor([[2, 3, 4,  ..., 5, 6, 7],
        [3, 2, 5,  ..., 6, 5, 4]])

In [16]:
slices[('Operator','Right','Operator')]['edge_index']

tensor([  0,   6,  10,  14,  18,  22,  28,  32,  36,  44,  52,  56,  60,  64,
         66,  68,  70,  74,  76,  82,  88,  92,  98, 100, 102, 104, 108, 110,
        112, 114, 118, 122, 126, 130, 144, 152, 154, 156, 158, 166, 172, 180,
        184, 186, 188, 190, 192, 196, 200, 204, 208, 216, 224, 230, 232, 234,
        238, 244, 248, 250, 252, 254, 256, 264, 266, 280, 282, 284, 286, 288,
        290, 300, 302, 304, 308, 312, 320, 328, 340, 344, 352, 360, 364, 390,
        394, 422, 426, 462, 466, 500, 506, 510, 512, 516, 522, 524, 532, 536,
        544, 596, 598, 600, 602, 610, 618, 624, 632, 636, 642, 646, 650, 654,
        660, 666, 672, 678, 690, 700, 710, 712, 714, 716, 718, 720, 724, 730,
        734, 738, 740, 744, 752, 754, 762, 764, 766, 768, 770, 774, 778, 788,
        792, 796, 806, 810, 814, 820])

In [72]:
data_list[220]

HeteroData(
  varid=[4],
  num_other_vars=4,
  y=[1, 22],
  NodeWithType={ x=[10, 53] },
  Operator={ x=[13, 64] },
  CallExpr={ x=[1, 141] },
  Literal={ x=[3, 53] },
  Default={ x=[3, 31] },
  (Operator, Left, NodeWithType)={ edge_index=[2, 6] },
  (Operator, Right, CallExpr)={ edge_index=[2, 1] },
  (CallExpr, Default, NodeWithType)={ edge_index=[2, 1] },
  (CallExpr, Param1, NodeWithType)={ edge_index=[2, 1] },
  (CallExpr, Param2, NodeWithType)={ edge_index=[2, 1] },
  (Operator, Right, NodeWithType)={ edge_index=[2, 4] },
  (NodeWithType, Default, Literal)={ edge_index=[2, 1] },
  (Operator, Right, Operator)={ edge_index=[2, 6] },
  (Operator, Left, Operator)={ edge_index=[2, 8] },
  (Operator, Right, Default)={ edge_index=[2, 2] },
  (Default, Default, Operator)={ edge_index=[2, 1] },
  (Default, ArrayVar, NodeWithType)={ edge_index=[2, 1] },
  (Operator, Left, Default)={ edge_index=[2, 1] },
  (Operator, Right, Literal)={ edge_index=[2, 1] },
  (Default, ArrayIdxExpr, Operator)

In [3]:
# loader = DataLoader(dataset, batch_size=4)
# loader
dataset[220]

IndexError: index 220 is out of bounds for dimension 0 with size 146

In [30]:
len(dataset)
dataset[232]

IndexError: index 232 is out of bounds for dimension 0 with size 226

In [5]:
# list(loader)[0]
list(DataLoader(dataset, batch_size=1))[0]

IndexError: index 133 is out of bounds for dimension 0 with size 133

In [30]:
dataset[0].num_edges

43

In [31]:
dataset[0].num_edge_features

{('Operator', 'Left', 'Default'): 0,
 ('Operator', 'Right', 'Literal'): 0,
 ('Operator', 'Right', 'Operator'): 0,
 ('Default', 'Default', 'Default'): 0,
 ('Default', 'Default', 'Operator'): 0,
 ('Default', 'rev_Left', 'Operator'): 0,
 ('Literal', 'rev_Right', 'Operator'): 0,
 ('Operator', 'rev_Default', 'Default'): 0}

In [34]:
dataset[0].edge_attrs()

['edge_index']

In [39]:
dataset[0].edge_stores[0].size()

TypeError: 'NoneType' object is not subscriptable

In [80]:
data = dataset[0]
dataset[25]

HeteroData(
  varid=[4],
  num_other_vars=[1],
  y=[1, 22],
  DeclRefExpr={ x=[2, 53] },
  BinaryOperator={ x=[1, 64] },
  IntegerLiteral={ x=[7, 53] },
  CStyleCastExpr={ x=[2, 53] },
  ParenExpr={ x=[6, 31] },
  (Operator, Left, Default)={ edge_index=[2, 1] },
  (Operator, Right, Literal)={ edge_index=[2, 1] },
  (Operator, Right, Operator)={ edge_index=[2, 4] },
  (Default, Default, Default)={ edge_index=[2, 1] },
  (Default, Default, Operator)={ edge_index=[2, 1] },
  (Default, rev_Left, Operator)={ edge_index=[2, 1] },
  (Literal, rev_Right, Operator)={ edge_index=[2, 1] },
  (Operator, rev_Default, Default)={ edge_index=[2, 1] }
)

In [60]:
import torch

for k, v in data.edge_index_dict.items():
    # print(v.shape[1], k)
    # print(torch.ones(1, v.shape[1]))
    data[k].edge_attr = torch.ones(1, v.shape[1])

In [74]:
# data.x_dict['Operator']
data['Operator']# = {'x':[]}
data

HeteroData(
  varid=[4],
  num_other_vars=[1],
  y=[1, 22],
  DeclRefExpr={ x=[3, 53] },
  BinaryOperator={ x=[8, 64] },
  IntegerLiteral={ x=[5, 53] },
  CStyleCastExpr={ x=[4, 53] },
  ParenExpr={ x=[2, 31] },
  Operator={},
  Default={},
  (Operator, Left, Default)={
    edge_index=[2, 8],
    edge_attr=[1, 8],
  },
  (Operator, Right, Literal)={
    edge_index=[2, 5],
    edge_attr=[1, 5],
  },
  (Operator, Right, Operator)={
    edge_index=[2, 6],
    edge_attr=[1, 6],
  },
  (Default, Default, Default)={
    edge_index=[2, 7],
    edge_attr=[1, 7],
  },
  (Default, Default, Operator)={
    edge_index=[2, 2],
    edge_attr=[1, 2],
  },
  (Default, rev_Left, Operator)={
    edge_index=[2, 8],
    edge_attr=[1, 8],
  },
  (Literal, rev_Right, Operator)={
    edge_index=[2, 5],
    edge_attr=[1, 5],
  },
  (Operator, rev_Default, Default)={
    edge_index=[2, 2],
    edge_attr=[1, 2],
  }
)

In [62]:
list(DataLoader([data]))[0]

/home/cls0027/envs/phd/lib/python3.8/site-packages/torch_geometric/data/storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set 'set()'. Please explicitly set 'num_nodes' as an attribute of 'data[Operator]' to suppress this warning
  warnings.warn(
/home/cls0027/envs/phd/lib/python3.8/site-packages/torch_geometric/data/storage.py:450: UserWarning: Unable to accurately infer 'num_nodes' from the attribute set 'set()'. Please explicitly set 'num_nodes' as an attribute of 'data[Default]' to suppress this warning
  warnings.warn(


RuntimeError: Could not infer dtype of NoneType